<a href="https://colab.research.google.com/github/chunabayashi0216/new_work/blob/main/tounyou_xgb_scaler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
pip install japanize-matplotlib

In [49]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

In [50]:
# Load in the train and test datasets
train_raw = pd.read_csv('/content/train.csv') #元の学習データ
test_raw = pd.read_csv('/content/test.csv') #元のテストデータ

In [51]:
print('The size of the df_train data:' + str(train_raw.shape))
print('The size of the df_test data:' + str(test_raw.shape))


The size of the df_train data:(3000, 10)
The size of the df_test data:(2000, 9)


In [53]:
df_train = train_raw.copy() #trainデータのコピー
df_test = test_raw.copy() #testデータのコピー

In [54]:
df_train['train_or_test'] = 'train' #trainデータの識別カラム追加 
df_test['train_or_test'] = 'test'   #testデータの 識別カラム追加  
df_test['Outcome'] = 9              #testデータに'Y'を仮置き

In [55]:
df_all = pd.concat([df_train,df_test],sort=False,axis=0).reset_index(drop=True)

In [56]:
df_all.head()

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,train_or_test
0,200,9,125,74,0,0,28.536910,0.444902,45,1,train
1,3832,4,109,80,0,0,28.047673,0.238243,22,0,train
2,4927,4,88,78,39,0,52.371341,0.279471,26,0,train
3,4088,9,125,74,0,0,40.062688,0.203922,45,0,train
4,3644,5,107,78,44,284,52.935068,0.284959,45,1,train



*   0	index				BMIint	インデックスとして使用
*   1	Pregnancies			int	妊娠した回数
*   2	Glucose				int	経口ブドウ糖負荷試験における2時間の血漿ブドウ糖濃度
*   3	BloodPressure			int	拡張期血圧
*   4	SkinThickness			int	皮膚のひだの厚さ
*   5	Insulin				int	血清インスリン
*   6	BMI				float	BMI
*   7	Age				int	年齢
*   8	DiabetesPedigreeFunction	float	糖尿病血統
*   9	Outcome				int	糖尿病であるか（糖尿病の場合1、でない場合0）

In [57]:
df_all.describe() # df_fullの要約統計量

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,2499.500000,3.567800,113.957600,68.752800,11.180000,11.741600,35.347125,0.400987,28.989400,3.743400
std,1443.520003,3.040535,21.333505,16.268147,14.232446,47.022061,7.496245,0.271619,8.509722,4.305121
min,0.000000,0.000000,57.000000,0.000000,0.000000,0.000000,0.000775,0.137377,21.000000,0.000000
25%,1249.750000,1.000000,100.000000,64.000000,0.000000,0.000000,32.453978,0.232290,22.000000,0.000000
50%,2499.500000,3.000000,111.000000,70.000000,0.000000,0.000000,33.828671,0.269691,26.000000,1.000000
75%,3749.250000,6.000000,125.000000,78.000000,24.000000,0.000000,39.627290,0.506604,33.000000,9.000000
max,4999.000000,13.000000,199.000000,110.000000,52.000000,744.000000,53.400629,2.302072,67.000000,9.000000


In [58]:
# 前処理
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   index                     5000 non-null   int64  
 1   Pregnancies               5000 non-null   int64  
 2   Glucose                   5000 non-null   int64  
 3   BloodPressure             5000 non-null   int64  
 4   SkinThickness             5000 non-null   int64  
 5   Insulin                   5000 non-null   int64  
 6   BMI                       5000 non-null   float64
 7   DiabetesPedigreeFunction  5000 non-null   float64
 8   Age                       5000 non-null   int64  
 9   Outcome                   5000 non-null   int64  
 10  train_or_test             5000 non-null   object 
dtypes: float64(2), int64(8), object(1)
memory usage: 429.8+ KB


In [61]:
# age education-num(mean = 0 std = 1 で正則化) -> age_sc education-num_sc
# 平均＝0、分散＝1　にスケーリング

from sklearn.preprocessing import StandardScaler

features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

for col in features:
    scaler = StandardScaler()
    df_all[col] = scaler.fit_transform(np.array(df_all[col].values).reshape(-1, 1))

df_all.head(5)

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,train_or_test
0,200,1.786772,0.517660,0.322577,-0.785608,-0.249729,-0.908574,0.161694,1.881636,1,train
1,3832,0.142160,-0.232409,0.691432,-0.785608,-0.249729,-0.973845,-0.599224,-0.821425,0,train
2,4927,0.142160,-1.216874,0.568481,1.954884,-0.249729,2.271260,-0.447421,-0.351327,0,train
3,4088,1.786772,0.517660,0.322577,-0.785608,-0.249729,0.629120,-0.725592,1.881636,0,train
4,3644,0.471083,-0.326167,0.568481,2.306229,5.790593,2.346468,-0.427214,1.881636,1,train


モデル構築

In [62]:
from sklearn.model_selection import train_test_split

# 最初に統合したtrainとtestを分離
train = df_all.query('train_or_test == "train"')
test = df_all.query('train_or_test == "test"')

train_feature = train.drop(columns=['Outcome', 'train_or_test'])
train_tagert = train['Outcome']
test_feature = test.drop(columns=['Outcome', 'train_or_test'])


In [63]:
test_feature.head(5)

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
3000,398,-1.173529,0.564539,0.691432,-0.785608,-0.249729,0.633599,1.552069,-0.938949
3001,3833,-0.186762,-1.216874,-0.538087,0.619773,-0.249729,0.595486,-0.724088,-0.821425
3002,4836,-0.186762,0.001988,0.445529,-0.785608,-0.249729,-0.286621,0.441925,-0.938949
3003,4572,-0.844607,1.502126,0.322577,-0.785608,-0.249729,-1.128260,0.382788,1.058965
3004,636,-0.844607,0.423901,1.306192,1.041387,2.727895,0.656790,1.471051,-0.116279


In [64]:
# 学習データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(
    train_feature, train_tagert, test_size=0.2, random_state=0, stratify=train_tagert )

In [65]:
print('train_feature = ', train_feature.shape)
print('test_feature = ', test_feature.shape)
print('train_tagert = ', train_tagert.shape)
print('X_train = ', X_train.shape)
print('X_test = ', X_test.shape)
print('y_train = ', y_train.shape)
print('y_test = ', y_test.shape)

train_feature =  (3000, 9)
test_feature =  (2000, 9)
train_tagert =  (3000,)
X_train =  (2400, 9)
X_test =  (600, 9)
y_train =  (2400,)
y_test =  (600,)


## xgboost(単体 パラメーターはDefult)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [66]:
import xgboost as xgb
from sklearn.metrics import accuracy_score


# パラメータ設定
params = {
    "objective": "binary:logistic",
    "evel_metric": "auc",
    "eta": 0.3,
    "max_depth": 6,                    
    "subsample": 1,                  
    "colsample_bytree":1 ,
    "silent" : 1
}

# 訓練・テストデータの設定
#
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# モデル作成
bst = xgb.train(params=params,
                  dtrain=dtrain,
                  num_boost_round=100,
                  early_stopping_rounds=10,
                  evals=[(dtest, 'test')]
                  ) 

# 検証用データが各クラスに分類される確率を計算する
preds = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)
# しきい値 0.5 で 0, 1 に丸める
pred = np.where(preds > 0.5, 1, 0)
# 精度 (Accuracy) を検証する
acc = accuracy_score(y_test, pred)
print('Accuracy:', acc)

[0]	test-error:0.208333
Will train until test-error hasn't improved in 10 rounds.
[1]	test-error:0.21
[2]	test-error:0.203333
[3]	test-error:0.216667
[4]	test-error:0.2
[5]	test-error:0.203333
[6]	test-error:0.196667
[7]	test-error:0.195
[8]	test-error:0.191667
[9]	test-error:0.195
[10]	test-error:0.198333
[11]	test-error:0.19
[12]	test-error:0.19
[13]	test-error:0.193333
[14]	test-error:0.193333
[15]	test-error:0.201667
[16]	test-error:0.2
[17]	test-error:0.2
[18]	test-error:0.2
[19]	test-error:0.198333
[20]	test-error:0.201667
[21]	test-error:0.195
Stopping. Best iteration:
[11]	test-error:0.19

Accuracy: 0.81


xgboost # 5分割交差検証

In [67]:
# train_feature, train_tagert を学習データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(
    train_feature, train_tagert, test_size=0.2, random_state=0, stratify=train_tagert )

# データを格納する
# テスト用
xgb_test = xgb.DMatrix(X_test, label=y_test)

# 5-fold CVモデルの学習
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)

# 5つのモデルを保存するリストの初期化
models = []
#accuracy、loglossを保存するNumPy配列の初期化
accuracies = np.array([])
loglosses = np.array([])

for fold_, (train_index, eval_index) in enumerate(kf.split(X_train, y_train)):    
    print(f'fold{fold_ + 1} start')
    X_train_cv = X_train.iloc[train_index]
    y_train_cv = y_train.iloc[train_index]
    X_eval_cv = X_train.iloc[eval_index]
    y_eval_cv = y_train.iloc[eval_index]
    
    # 学習用
    xgb_train = xgb.DMatrix(X_train_cv, label=y_train_cv)

    # 検証用
    xgb_eval = xgb.DMatrix(X_eval_cv, label=y_eval_cv)
   
    
    # パラメータを設定(defult)
    xgb_params = {
        "objective": "binary:logistic",
        "evel_metric": "auc",
        "eta": 0.3,
        "max_depth": 6,                    
        "subsample": 1,                  
        "colsample_bytree":1 ,
        "silent" : 1
    }  
    
    # 学習
    evals = [(xgb_train, 'train'), (xgb_eval, 'eval')] # 学習に用いる検証用データ
    evaluation_results = {}                            # 学習の経過を保存する箱
    bst = xgb.train(xgb_params,                        # 上記で設定したパラメーター
                    xgb_train,                         # 使用するデータセット
                    num_boost_round=500,               # 学習の回数
                    early_stopping_rounds=10,          # アーリーストッピング
                    evals=evals,                       # 上記で設定した検証用データ
                    evals_result=evaluation_results,   # 上記で設定した箱
                    verbose_eval=0                     # 学習の経過の表示(非表示)
                    ) 
    
    # テストデータで予測する
    y_pred = bst.predict(xgb_test, ntree_limit=bst.best_ntree_limit)
        
    # しきい値 0.5 で 0, 1 に丸める
    pred = np.where(y_pred > 0.5, 1, 0)
    
    # 精度 (Accuracy) を検証する
    accuracy = accuracy_score(y_test, pred)

    # Logloss を計算する
    logloss = log_loss(y_test, y_pred)
    
    print('accuracy:', accuracy, 'logloss: ', logloss)
    
    # 学習が終わったモデルをリストに入れておく
    models.append(bst) 
    # 学習結果をNumPy配列に入れておく
    accuracies = np.append(accuracies, accuracy)
    loglosses = np.append(loglosses, logloss)
    
# 正解ラベルの確率とLogloss（平均値）
print('accuracy_ave: ', np.mean(accuracies), 'logloss_ave: ', np.mean(loglosses)) 

fold1 start
accuracy: 0.795 logloss:  0.44968849395091337
fold2 start
accuracy: 0.7916666666666666 logloss:  0.4390248459290403
fold3 start
accuracy: 0.8 logloss:  0.4425261436961591
fold4 start
accuracy: 0.785 logloss:  0.48833803938216686
fold5 start
accuracy: 0.79 logloss:  0.4570431795498977
accuracy_ave:  0.7923333333333333 logloss_ave:  0.45532414050163544


In [68]:
#グリッドサーチでよいパラメータを調べる　例として'max_depth'　'eta'

gridsearch_params = [
    (max_depth, eta)
    for max_depth in [3, 4, 5, 6, 7, 8, 9, 10]
    for eta in [0.05, 0.04, 0.03, 0.02, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5]
]
gridsearch_params

[(3, 0.05),
 (3, 0.04),
 (3, 0.03),
 (3, 0.02),
 (3, 0.01),
 (3, 0.1),
 (3, 0.2),
 (3, 0.3),
 (3, 0.4),
 (3, 0.5),
 (4, 0.05),
 (4, 0.04),
 (4, 0.03),
 (4, 0.02),
 (4, 0.01),
 (4, 0.1),
 (4, 0.2),
 (4, 0.3),
 (4, 0.4),
 (4, 0.5),
 (5, 0.05),
 (5, 0.04),
 (5, 0.03),
 (5, 0.02),
 (5, 0.01),
 (5, 0.1),
 (5, 0.2),
 (5, 0.3),
 (5, 0.4),
 (5, 0.5),
 (6, 0.05),
 (6, 0.04),
 (6, 0.03),
 (6, 0.02),
 (6, 0.01),
 (6, 0.1),
 (6, 0.2),
 (6, 0.3),
 (6, 0.4),
 (6, 0.5),
 (7, 0.05),
 (7, 0.04),
 (7, 0.03),
 (7, 0.02),
 (7, 0.01),
 (7, 0.1),
 (7, 0.2),
 (7, 0.3),
 (7, 0.4),
 (7, 0.5),
 (8, 0.05),
 (8, 0.04),
 (8, 0.03),
 (8, 0.02),
 (8, 0.01),
 (8, 0.1),
 (8, 0.2),
 (8, 0.3),
 (8, 0.4),
 (8, 0.5),
 (9, 0.05),
 (9, 0.04),
 (9, 0.03),
 (9, 0.02),
 (9, 0.01),
 (9, 0.1),
 (9, 0.2),
 (9, 0.3),
 (9, 0.4),
 (9, 0.5),
 (10, 0.05),
 (10, 0.04),
 (10, 0.03),
 (10, 0.02),
 (10, 0.01),
 (10, 0.1),
 (10, 0.2),
 (10, 0.3),
 (10, 0.4),
 (10, 0.5)]

グリッドサーチでパラメーター探索

In [69]:
min_logless = float('Inf')
min_score = float('Inf')

best_param = []
min_train_error_mean = float('Inf')
min_test_error_mean = float('Inf')


for max_depth, eta in gridsearch_params:
    print('max_depth={}, eta={}'.format(max_depth, eta))
    
    params['max_depth'] = max_depth
    params['eta'] = eta
    
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=1000, 
        seed=0,
        nfold=5,
 #       metrics={'rmse'},
        early_stopping_rounds=5
    )
    
    train_error_mean = cv_results['train-error-mean'].min()
    test_error_mean = cv_results['test-error-mean'].min()
    boost_rounds = cv_results['train-error-mean'].argmin()
    print('train-error-mean {} train-error-mean {} for {} rounds'.format(train_error_mean, test_error_mean, boost_rounds))
    if train_error_mean < min_train_error_mean:
        min_train_error_mean = train_error_mean
        min_test_error_mean = test_error_mean
        best_param = (max_depth, eta)
    
print('Best params {}, train_error_mean {},  test_error_mean {}'.format(best_param, train_error_mean, test_error_mean))

max_depth=3, eta=0.05
train-error-mean 0.193854 train-error-mean 0.21416659999999998 for 25 rounds
max_depth=3, eta=0.04
train-error-mean 0.2160418 train-error-mean 0.22625 for 0 rounds
max_depth=3, eta=0.03
train-error-mean 0.213125 train-error-mean 0.2245832 for 5 rounds
max_depth=3, eta=0.02
train-error-mean 0.2160418 train-error-mean 0.22625 for 0 rounds
max_depth=3, eta=0.01
train-error-mean 0.2160418 train-error-mean 0.22625 for 0 rounds
max_depth=3, eta=0.1
train-error-mean 0.18583319999999998 train-error-mean 0.20541679999999998 for 28 rounds
max_depth=3, eta=0.2
train-error-mean 0.1760416 train-error-mean 0.2029166 for 19 rounds
max_depth=3, eta=0.3
train-error-mean 0.16020859999999998 train-error-mean 0.20249999999999999 for 19 rounds
max_depth=3, eta=0.4
train-error-mean 0.15677080000000002 train-error-mean 0.205 for 16 rounds
max_depth=3, eta=0.5
train-error-mean 0.1435416 train-error-mean 0.2062498 for 18 rounds
max_depth=4, eta=0.05
train-error-mean 0.1679166 train-error-

xgboost # 5分割交差検証 ベストパラメーター　max_depth=10, eta=0.4

In [70]:
# train_feature, train_tagert を学習データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(
    train_feature, train_tagert, test_size=0.2, random_state=0, stratify=train_tagert )

# データを格納する
# テスト用
xgb_test = xgb.DMatrix(X_test, label=y_test)

# 5-fold CVモデルの学習
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)

# 5つのモデルを保存するリストの初期化
models = []
#accuracy、loglossを保存するNumPy配列の初期化
accuracies = np.array([])
loglosses = np.array([])

for fold_, (train_index, eval_index) in enumerate(kf.split(X_train, y_train)):    
    print(f'fold{fold_ + 1} start')
    X_train_cv = X_train.iloc[train_index]
    y_train_cv = y_train.iloc[train_index]
    X_eval_cv = X_train.iloc[eval_index]
    y_eval_cv = y_train.iloc[eval_index]
    
    # 学習用
    xgb_train = xgb.DMatrix(X_train_cv, label=y_train_cv)

    # 検証用
    xgb_eval = xgb.DMatrix(X_eval_cv, label=y_eval_cv)
   
    
    # パラメータを設定(defult)
    xgb_params = {
        "objective": "binary:logistic",
        "evel_metric": "auc",
        "eta":0.5,
        "max_depth": 10,                    
        "subsample": 1,                  
        "colsample_bytree":1 ,
        "silent" : 1
    }  
    
    # 学習
    evals = [(xgb_train, 'train'), (xgb_eval, 'eval')] # 学習に用いる検証用データ
    evaluation_results = {}                            # 学習の経過を保存する箱
    bst = xgb.train(xgb_params,                        # 上記で設定したパラメーター
                    xgb_train,                         # 使用するデータセット
                    num_boost_round=500,               # 学習の回数
                    early_stopping_rounds=10,          # アーリーストッピング
                    evals=evals,                       # 上記で設定した検証用データ
                    evals_result=evaluation_results,   # 上記で設定した箱
                    verbose_eval=0                     # 学習の経過の表示(非表示)
                    ) 
    
    # テストデータで予測する
    y_pred = bst.predict(xgb_test, ntree_limit=bst.best_ntree_limit)
        
    # しきい値 0.5 で 0, 1 に丸める
    pred = np.where(y_pred > 0.5, 1, 0)
    
    # 精度 (Accuracy) を検証する
    accuracy = accuracy_score(y_test, pred)

    # Logloss を計算する
    logloss = log_loss(y_test, y_pred)
    
    print('accuracy:', accuracy, 'logloss: ', logloss)
    
    # 学習が終わったモデルをリストに入れておく
    models.append(bst) 
    # 学習結果をNumPy配列に入れておく
    accuracies = np.append(accuracies, accuracy)
    loglosses = np.append(loglosses, logloss)
    
# 正解ラベルの確率とLogloss（平均値）
print('accuracy_ave: ', np.mean(accuracies), 'logloss_ave: ', np.mean(loglosses)) 

fold1 start
accuracy: 0.79 logloss:  0.4706345662754029
fold2 start
accuracy: 0.7966666666666666 logloss:  0.4873944215104469
fold3 start
accuracy: 0.795 logloss:  0.49681223545145864
fold4 start
accuracy: 0.7716666666666666 logloss:  0.4881284616018335
fold5 start
accuracy: 0.7516666666666667 logloss:  0.5045263815050324
accuracy_ave:  0.781 logloss_ave:  0.48949921326883483
